In [ ]:
import openturns as ot
import numpy as np
import math
ot.RandomGenerator.SetSeed(2026)


## 1

In [ ]:
j = 28
A = j**3 / (32 - j)
a = np.pi / j
N = 200

X = ot.Uniform(- A, A)
Y = ot.Uniform(- A, A)
T = ot.Uniform(- a, a)

distribution = ot.ComposedDistribution([X, Y, T])

def g(X):
    x, y, t = X
    return [np.cos(t)*x + np.sin(t)*y]

# g = ot.ComposedDistribution(["X","Y","T"],["cos(T)*x"+sin(T)*y])

sample = distribution.getSample(N)
X_sample = sample[:, 0]
Y_sample = sample[:, 1]
T_sample = sample[:, 2]

model = ot.PythonFunction(3, 1, g)

Y_out = model(sample)


In [ ]:
sobol = ot.SaltelliSensitivityAlgorithm(
    distribution, model, N
)

S_first = sobol.getFirstOrderIndices()
S_total = sobol.getTotalOrderIndices()

print(S_first)
print(S_total)

## 3

In [ ]:
X_train = sample
Y_train = Y_out

basis = ot.ConstantBasisFactory(3).build()
covariance_model = ot.SquaredExponential([1.0]*3, [1.0])
algo = ot.KrigingAlgorithm(X_train, Y_train, covariance_model, basis)
algo.run()
Kriging_result = algo.getResult()
Kriging_meta = Kriging_result.getMetaModel()

## 4

In [ ]:
sensitivity_meta = ot.SaltelliSensitivityAlgorithm(distribution, Kriging_meta)
first_order_meta = sensitivity_meta.getFirstOrderIndices()
total_order_meta = sensitivity_meta.getTotalOrderIndices()

print(first_order_meta)
print(total_order_meta)

## 5

In [ ]:
import matplotlib.pyplot as plt

N_value = [100, 200, 500, 1000]
S1, ST = [], []

for N in N_value:
    sample = distribution.getSample(N)
    Y_out = model(sample)
    sa = ot.SaltelliSensitivityAlgorithm(sample, Y_out, distribution, model)
    S1.append(sa.getFirstOrderIndices())
    ST.append(sa.getTotalOrderIndices())

S1 = np.array(S1)
ST = np.array(ST)

plt.plot(N_value, S1[:,0], label = "S1(X)")
plt.plot(N_value, ST[:,0], label = "ST(X)")
plt.legend()
plt.xlabel("N")
plt.ylabel("Indice de Sobol")
plt.show()

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import spearmanr


j = 15
A = (j**3) / (32 - j)
bounds = np.array([[-A, A], [-A, A], [-np.pi/j, np.pi/j]])


def g(data): return np.cos(data[:, 2]) * data[:, 0] + np.sin(data[:, 2]) * data[:, 1]

# 计算 Sobol 指数 (基于元模型)
def compute_sobol(gp_model, n_mc=10000):

    M1 = np.random.uniform(bounds[:, 0], bounds[:, 1], (n_mc, 3))
    M2 = np.random.uniform(bounds[:, 0], bounds[:, 1], (n_mc, 3))
    Y_M1 = gp_model.predict(M1)
    var_Y = np.var(Y_M1)
    
    indices = []
    for i in range(3):
        M_mix = M2.copy(); M_mix[:, i] = M1[:, i] # 混合矩阵
        Y_mix = gp_model.predict(M_mix)
        # Sobol
        Si = np.mean(Y_M1 * (Y_mix - np.mean(Y_M1))) / var_Y
        indices.append(Si)
    return indices

#
print(f"{'N':<5} | {'X-Corr (Q2)':<12} | {'X-Sobol (Q4)':<12} | {'Y-Sobol':<12} | {'T-Sobol':<12}")
print("-" * 65)

for N in [200, 50, 100, 500]:
    # Q1
    X_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], (N, 3))
    Y_sample = g(X_sample)

    # Q2
    corrs = [abs(spearmanr(X_sample[:, i], Y_sample)[0]) for i in range(3)]

    # Q3: Kriging
    kernel = C(1.0) * RBF([10, 10, 1])
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, random_state=42)
    gp.fit(X_sample, Y_sample)

    # Q4
    sobols = compute_sobol(gp)

    # 
    prefix = "-->" if N == 200 else "   "
    print(f"{prefix} {N:<3} | {corrs[0]:.4f}       | {sobols[0]:.4f}       | {sobols[1]:.4f}       | {sobols[2]:.4f}")



In [ ]:
import openturns as ot
import numpy as np

# 参数
j = 15
A = j**3 / (32 - j)

# 输入分布
X = ot.Uniform(-A, A)
Y = ot.Uniform(-A, A)
T = ot.Uniform(-np.pi/j, np.pi/j)
distribution = ot.ComposedDistribution([X, Y, T])

# 模型
model = ot.SymbolicFunction(
    ["X", "Y", "T"],
    ["cos(T)*X + sin(T)*Y"]
)

# 样本
N = 200
input_sample = distribution.getSample(N)
output_sample = model(input_sample)

In [ ]:
sobol_q2 = ot.SobolIndicesAlgorithm(input_sample, output_sample, input_sample.getSize())
S1_q2 = sobol_q2.getFirstOrderIndices()
ST_q2 = sobol_q2.getTotalOrderIndices()

In [ ]:

# 基函数
basis = ot.ConstantBasisFactory(3).build()

# 协方差模型
cov = ot.SquaredExponential([1.0, 1.0, 0.3])

# Kriging
algo = ot.KrigingAlgorithm(input_sample, output_sample, cov, basis)
algo.run()
kriging_model = algo.getResult().getMetaModel()

In [ ]:

sobol_q4 = ot.SaltelliSensitivityAlgorithm(distribution, kriging_model)
sobol_q4.setSize(2000)  # Sobol 需要较大样本
sobol_q4.run()

S1_q4 = sobol_q4.getFirstOrderIndices()
ST_q4 = sobol_q4.getTotalOrderIndices()

In [ ]:
for N in [50, 100, 200, 500]:
    input_sample = distribution.getSample(N)
    output_sample = model(input_sample)

    sobol = ot.SobolIndicesAlgorithm(input_sample, output_sample)
    print(f"N={N}, S1={sobol.getFirstOrderIndices()}")

In [2]:
import openturns as ot
import numpy as np

# ==================================================
# 参数
# ==================================================
j = 15
A = j**3 / (32 - j)

# ==================================================
# 输入分布
# ==================================================
X = ot.Uniform(-A, A)
Y = ot.Uniform(-A, A)
T = ot.Uniform(-np.pi/j, np.pi/j)
distribution = ot.ComposedDistribution([X, Y, T])

# ==================================================
# 模型
# ==================================================
model = ot.SymbolicFunction(
    ["X", "Y", "T"],
    ["cos(T)*X + sin(T)*Y"]
)

# ==================================================
# Question 1 — 采样 + 输出
# ==================================================
N = 200
input_sample = distribution.getSample(N)
output_sample = model(input_sample)

print("Q1 done")

# ==================================================
# Question 2 — 样本敏感性（Spearman）
# ==================================================
corr = ot.CorrelationAnalysis(input_sample, output_sample)
spearman = corr.computeSpearmanCorrelation()

print("Q2 — Spearman:", spearman)

# ==================================================
# Question 3 — Kriging 元模型
# ==================================================
basis = ot.ConstantBasisFactory(3).build()
cov = ot.SquaredExponential([1.0, 1.0, 0.3])

kriging_algo = ot.KrigingAlgorithm(
    input_sample,
    output_sample,
    cov,
    basis
)
kriging_algo.run()
kriging_model = kriging_algo.getResult().getMetaModel()

print("Q3 done")

# ==================================================
# Question 4 — Sobol (Saltelli + méta-modèle)
# ==================================================
N_sobol = 1000

# Saltelli 结构化实验
experiment = ot.SaltelliExperiment(distribution, N_sobol)
X_sobol = experiment.generate()

# 用元模型计算输出
Y_sobol = kriging_model(X_sobol)

# Sobol 指数
sobol_q4 = ot.SobolIndicesAlgorithm(X_sobol, Y_sobol, N_sobol)

S1 = sobol_q4.getFirstOrderIndices()
ST = sobol_q4.getTotalOrderIndices()

print("Q4 — Sobol S1:", S1)
print("Q4 — Sobol ST:", ST)

# ==================================================
# Question 5 — 改变 N 的影响（基于 Spearman）
# ==================================================
for N in [50, 100, 200, 500]:
    input_sample = distribution.getSample(N)
    output_sample = model(input_sample)

    corr = ot.CorrelationAnalysis(input_sample, output_sample)
    spearman = corr.computeSpearmanCorrelation()

    print(f"Q5 — N={N}, Spearman={spearman}")

Q1 done
Q2 — Spearman: [0.989789,0.111565,-0.0366444]
Q3 done


AttributeError: module 'openturns' has no attribute 'SaltelliExperiment'